## Chapter 3 - Iterative modeling



### README



Improving quality of the code and quality of the model.

-   Iterating the bike share model
-   Using more than one `State` object
-   Adding docstrings
-   Dealing with negative bikes
-   Introducing comparison operators
-   Introducing metrics
-   Summary and exercises



### Iterative modeling



Iterative modeling means starting with a simple model, identifying the
most important issues for the problem setting, and making gradual
improvements.

What assumptions is the bike share model based on? In which ways is it
unrealistic - what are the differences between the model and the real
world?

1.  Customer arrival probability varies over time.
2.  Travel time between bike stations is not zero.
3.  Bike availability is not guaranteed.
4.  LEAP and Community Centre look completely different.
5.  There are more customers at the Community Centre.
6.  Customers come in different types.
7.  Bikes come in different types.
8.  Checking out bikes might cost money.

Are any of these more relevant than others? Relevance depends on the
purpose for the modeling simulation. What is our likely purpose?

-   Understand the system with its elements, relationships, boundary and
    the resulting states.

For this purpose, specific details about the elements (e.g. type of
bike or type of customer) could be important once we've understood the
basic dynamics (i.e. evolution over time).

The different looks (and location in town) of our stations might
affect the arrival/checkout probability: more students at LEAP but
more people overall at the Community Centre.

Types of customer is important if we're trying to make a prediction by
customer type - like "which age group is most likely to check out a
bike?" Same for the bike type - which bikes should we buy?

The first assumption (time dependency) can be neglected if we simulate
over a short period of time (like one hour).

The second assumption (travel time between stations) might not affect
the results very much depending on the distance between stations - but
if we're after the impact of this feature on the dynamic of the model,
we have to add it.

The third assumption is important because negative bike numbers are an
obvious violation of the physical laws and just don't make sense.



### Using more than one `State` object



What if there is more than one bike share system, or if we want to run
more than one simulation? We need to be able to define more than one
set of state variables.

Import `modsim`, then redefine the update functions:



In [1]:
# import State from modsim
from modsim import State, show, source_code, flip, TimeSeries

# redefine `bike_to_city()` to take the argument `state`
def bike_to_city(state):
    print("Moving bike to city Community Center.")
    state.leap -= 1
    state.city += 1

# redefine `bike_to_city()` to take the argument `state`
def bike_to_leap(state):
    print("Moving bike to Lyon's LEAP.")
    state.leap += 1
    state.city -= 1

The value of `state` could be any `State` object not just `bikeshare`. When
we now update the bike stations, we need to supply a `State` object:



In [1]:
# Initialize a `bikeshare` State
bikeshare = State(city=10,leap=2)

# Move a bike to city
bike_to_city(bikeshare)

# Show the state of `bikeshare`
print(show(bikeshare))

Moving bike to city Community Center.
      state
city     11
leap      1

The update call has the same effect as:

    state = bikeshare
    state.leap -= 1
    state.city += 1

Notice the difference between the `state` parameter in the update
function, which accepts `State` arguments.

Now we can create as many `State` objects as we want, and update them
independently:



In [1]:
# define two states, `bikeshare1` and `bikeshare2`
bikeshare1 = State(city=10,leap=2)
bikeshare2 = State(city=2,leap=10)

# Move a bike to city in both states and show them
bike_to_city(bikeshare1)
print(show(bikeshare1))
bike_to_city(bikeshare2)
print(show(bikeshare2))

Moving bike to city Community Center.
      state
city     11
leap      1
Moving bike to city Community Center.
      state
city      3
leap      9

Changes in either state do not affect the other. Now we can simulate
different bike share systems (e.g. different initial conditions), or
run multiple simulations.



### Adding docstrings



There are two types of in-program documentation:

1.  comments (use liberally) after `#`
2.  docstrings (use strategically) between `"""`

In IPython, you get the docstring when asking for help with `?`.

Add the following documentation to the update functions by printing
the text between two sets of `"""` marks on the next line after the `def`
line, then print the docstring:

    """Move one bike from [station1] to [station2]
       state: bikeshare State object
    """



In [1]:
def bike_to_city(state):
    """Move one bike from LEAP to city community center.

    state: bikeshare State object
    """
    print("Moving bike to city Community Center.")
    state.leap -= 1
    state.city += 1

def bike_to_leap(state):
    """Move one bike from city community center to LEAP.

    state: bikeshare State object
    """
    print("Moving bike to Lyon's LEAP.")
    state.leap += 1
    state.city -= 1

Calling the help with `help`:



In [1]:
help(bike_to_city)
help(bike_to_leap)

#+begin_example
Help on function bike_to_city in module __main__:

bike_to_city(state)
    Move one bike from LEAP to city community center.

    state: bikeshare State object

Help on function bike_to_leap in module __main__:

bike_to_leap(state)
    Move one bike from city community center to LEAP.

    state: bikeshare State object
#+end_example

What happens if you enter `help(bike_to_city())` and
`help(bike_to_city(state))` - explain the errors!



### Scope and keyword parameter



You can call the function `updateState` without argument because the
function definition has a default value.



In [1]:
from modsim import State, show

def updateState(state=State(variable=10)):
    state.variable -= 1
    return print(show(state))

updateState()

state
variable      9

If you return `state` from within the function, it will not be known
outside unless



In [1]:
from modsim import State, show

def updateState(state=State(variable=10)):
    state.variable -= 1
    return state

Zustand = updateState()
print(show(Zustand))
print(state)

As an exercise, make this last block safe for `print(state)`, for the
call of an unknown state:



In [1]:
from modsim import State, show

def updateState(state=State(variable=10)):
    state.variable -= 1
    return state

Zustand = updateState()
print(show(Zustand))
try:
    print(state)
except NameError:
    print("`state` is not known")

### Dealing with negative bikes



To account for negative bikes, add a conditional statement to the
update function and test if the update would lead to a negative
`state.leap`, the state at the LEAP station:



In [1]:
def bike_to_city(state):
    if state.leap == 0:
    return  # no return value, or rather `None`

    state.city += 1
    state.leap -= 1

Test this function for zero and non-zero values of `state.city`:



In [1]:
## ten bikes at city, 2 bikes at leap
bikeshare1 = State(city=10,leap=2)
bike_to_city(bikeshare1)
print(show(bikeshare1))

## all bikes at city, no bikes at leap
bikeshare2 = State(city=12,leap=0)
bike_to_city(bikeshare2)  # this would lead to leap = -1
print(show(bikeshare2))

state
city     11
leap      1
      state
city     12
leap      0

When a function encounters a `return` statement with or without return
value, it ends immediately ignoring the remainder of its statements.

Actually, the story is more complicated: an empty `return` statement
returns a value `None`. Can you show this by writing a function? What
data type is `None`?



In [1]:
def returnDemo():
    # functions don't HAVE to do something
    return

r = returnDemo()
print(r)
print(type(None))

None
<class 'NoneType'>

So `None` is of the type `NoneType` (there is no other variable in that
class). It represents a missing value. It's not equivalent to `0` or
`False` or an empty data structure like an empty list `[]`.

No more negative bikes at LEAP. Write an update function so that at
the city community center we will also never have negative bikes:



In [1]:
def bike_to_leap(state):
    if state.city == 0:
        return
    state.leap += 1
    state.city -= 1

Test the function `bike_to_leap` for negative bikes at LEAP:



In [1]:
## 7 bikes at leap, 5 bikes at city
bikeshare1 = State(leap=7,city=5)
bike_to_leap(bikeshare1)
print(show(bikeshare1))

## all 12 bikes at leap, no bikes at city
bikeshare2 = State(leap=12,city=0)
bike_to_leap(bikeshare2)
print(show(bikeshare2))

state
leap      8
city      4
      state
leap     12
city      0

### Using comparison operators



Don't confuse the assignment operator `=` with the comparison operator
`==`. The former distributes computer memory to variables and spits out
nothing while the latter checks equality and spits out a Boolean
value.

The assignment operator does, the comparison operator does not change
the value of its left hand side argument:



In [1]:
x = 5  # changes value of x
print(x)
print(x == 5)  # does not change value of x

5
True

What error do you get if you mess this up in a conditional statement?

1.  Set x to 5, check `if` it is 5 and `print` "x is 5" (use an f-string).
2.  Now assign x to 5 in the conditional statement and run it again.



In [1]:
x = 5
if x == 5:
    print(f'x is {x}')

x is 5

Write a conditional statement that checks if x is NOT 5 using the
comparison operator `!=`. Start by assigning x to 6 and print "x is not
5 but 6". Use an f-string.



In [1]:
x = 6
y = 5
if x != y:
    print(f'x is not {y} but {x}')

x is not 5 but 6

To check for `None`, do not use `==` but the `is` keyword:



In [1]:
x = None
if x is None:
    print("x is None!")

x is None!

The comparison operator will not lead to an error or an exception but
to logical errors because `==` checks for **equality** whereas `is` checks for
**identity** - it's better practice to use the latter to avoid unexpected
behavior.



### Defining simulation metrics



Our current `step` function model is *stochastic* because the arrival of
customers is dictated by a randomized draw with varying
probabilities using `flip` - it's different every time.

The opposite extreme would be a *deterministic* model that does the same
thing every time the model runs. Mathematical models that are based on
evaluating closed mathematical expressions are deterministic by nature.

**Could we build a deterministic model for the bike share system?** Yes,
we could, if we could figure out all the rules that affect our
elements (customers, stations) and their relationships.

**Which type of model is better?** It depends on the system. The
deterministic model makes exact predictions but cannot handle
uncertainty (sudden change of rules), while the stochastic model
handles uncertainty well but is stable only for large numbers.

**What would make the model better for the customer?** For example knowing
the probability of finding an available bike (to minimize the chance
of not getting one, or maximize the chance of getting one).

**What would make the model better for the owner?** For example knowing if
a customer got a bike or not (to minimize the number of customers who
don't get a bike or maximize the number of bikes in use).

Statistics that quantify how well the system works are simulation
*metrics*. "How well" is relative to the purpose of the
simulation. Applying metrics boils down to optimization.



### Measuring the number of unhappy customers



When modeling systems with human participation, we often (except in
binary situations like life or death) need to use indirect measures,
e.g. we cannot know if a customer is really unhappy - but we can infer
unhappiness if he arrives at a station and finds no bike available.

Being forced to quantify the unquantifiable has a good and a bad
side. The good side is that it provides reliability - e.g. it enables
us to send a man to the moon based on building a rocket for "everyman"
instead of exactly for the person who will sit in that rocket. The bad
side is that it removes degrees of freedom from the system:
"un/happiness" is a state of infinite gradation, a continuum that is
rarely stable and usually changes from moment to moment. But to be
able to compute things, we must simplify and freeze feelings.

Here is a version of `bike_to_leap` that keeps track of the number of
unhappy customers, measured in the number of customers who arrive at a
station with no bikes:



In [1]:
def bike_to_leap(state):
    if state.city == 0:
        state.city_empty += 1  # add one unhappy customer
        return
    state.leap += 1
    state.city -= 1

We have essentially added another state variable to our system. We
need to add it whenever we create a `State` object:



In [1]:
bikeshare = State(leap = 12,
                  city = 0,
                  city_empty = 0)  # initialize with 0 unhappy customers

We test it by calling `bike_to_leap` (withdrawing a bike from `city`):



In [1]:
print(show(bikeshare))
bike_to_leap(bikeshare)
print(show(bikeshare))

state
leap           12
city            0
city_empty      0
            state
leap           12
city            0
city_empty      1

Now we have one unhappy customer at the city station.



### Concept and code summary



Concept:

-   Iterative modeling means starting with a simple model, identifying
    the most important issues for the problem setting, and making
    gradual improvements.
-   Variables are local to functions, and parameters can have defaults
    when using keywords for them.
-   *Stochastic* models produce different outcomes each time they are run,
    *deterministic* models produce the same outcome each time they're run.
-   Statistics that measure how well the system works are called *metrics*.

Code:

-   We used a conditional operator, `==`, to check whether a bike is
    available, in order to avoid negative bikes.
-   We added a parameter, `state`, so we can work with more than one `State`
    object.
-   We added a docstring that explains how to use a function and a
    comment that explains how it works.
-   We added a state variable, `city_empty`, to count the number of
    unhappy customers, which is a metric we’ll use to quantify how well
    the system works.



### Exercises 3 (30 points)



1.  `*` means easy (10 points)
2.  `**` means medium (20 points)
3.  `***` means hard (30 points)



#### Add docstrings (`*`)



Before starting the exercise, make sure that you have all of the code
written so far in one place:

1.  `run_simulation(state,p1,p2,num_steps)`
2.  `step(state,p1,p2)`
3.  `bike_to_city(state)`
4.  `bike_to_leap(state)`

Complete the function definitions with docstrings, which includes a
short description of what the function does followed by a list of
parameters explained.

Solution:



#### Unhappy customers at the LEAP station (`*`)



Modify the update function `bike_to_city` so that it checks whether a
bike is available at LEAP. If not, it should add `1` to `leap_empty`
(equivalent to one unhappy customer at the LEAP station).

To test the function, create a `State` that initializes `leap` and
`leap_empty` to `0`, run `bike_to_city` and check the result.

Solution:



In [1]:
## definition
def bike_to_city(state):
    if state.leap == 0:
        state.leap_empty += 1
        return
    state.leap -= 1
    state.city += 1

## initialization
bikeshare = State(leap = 0,
                  city = 12,
                  leap_empty = 0)

## testing
print(show(bikeshare))
bike_to_city(bikeshare)
print(show(bikeshare))

state
leap            0
city           12
leap_empty      0
            state
leap            0
city           12
leap_empty      1

#### Run a simulation without negative bikes (`*`)



Run the simulation with parameters `p1=0.3`, `p2=0.2`, and `num_steps=60`,
and confirm that the number of bikes is never negative.

Start with this initial state: 10 bikes at city, 2 bikes at LEAP, and
no unhappy customers at either location.

Solution:

